In [1]:
# import requests
# import json
# import threading
# import time
# import os  # Voor het maken van directories
# from datetime import datetime

# start_data = "2023-01-01"
# end_date = "2025-12-31"
# Locatie = "Rotterdam"
# species_per_batch = 50  # Aantal soorten per batch

# # Genereer een unieke directory naam met een timestamp
# timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# batch_directory = f"batches_{timestamp}"

# # Maak de directory aan, zelfs als deze al bestaat
# os.makedirs(batch_directory, exist_ok=True)



# # Het species_id vanaf waar je wilt starten
# start_species_id = 1  # Pas dit aan naar het gewenste start species_id



# # Functie om observaties op te halen voor een bepaald soort
# def fetch_observations_for_species(species_id, results, no_data_count_lock):
#     API_URL = f"https://observation.org/api/v1/species/{species_id}/observations/?limit=1000&country_id=166&date_after={start_data}&date_before={end_date}&user=&location={Locatie}"
    
#     response = requests.get(API_URL)
#     if response.status_code != 200:
#         print(f"Fout bij ophalen voor soort {species_id}: {response.status_code}")
#         return
    
#     data = response.json()
#     # Controleer of de 'detail' in de response de foutmelding bevat
#     if 'detail' in data and data['detail'] == "No Species matches the given query.":
#         print(f"Geen observaties voor soort {species_id}.")
#         return
    
#     # Voeg de resultaten toe aan de lijst
#     with results['lock']:
#         results['observations'].extend(data.get("results", []))

#     # Wacht om rate-limiting te voorkomen
#     time.sleep(5)  # Wacht 1 seconde om rate limit te voorkomen

# # Functie die de threading afhandelt
# def fetch_all_observations():
#     # Opslaan van alle observaties in één bestand
#     all_observations = {'observations': [], 'lock': threading.Lock()}
#     threads = []
#     species_id = start_species_id  # Begin vanaf de gewenste species_id
#     batch_count = 1  # Huidige batch nummer
#     species_in_batch = 0  # Tellen hoeveel soorten er in de huidige batch zitten
    
#     while True:
#         # Start een nieuwe thread voor het ophalen van observaties voor dit soort
#         print(f"Verwerken voor soort {species_id}...")
#         thread = threading.Thread(target=fetch_observations_for_species, args=(species_id, all_observations, threading.Lock()))
#         threads.append(thread)
#         thread.start()
        
#         species_id += 1
#         species_in_batch += 1
        
#         # Wacht totdat 15 threads zijn afgerond of de batch 50 soorten heeft bereikt
#         if len(threads) >= 15 or species_in_batch >= species_per_batch:
#             for t in threads:
#                 t.join()  # Wacht tot alle threads in de batch zijn afgerond
#             threads = []  # Reset de threads lijst om ruimte te maken voor nieuwe

#             # Sla de observaties op in een nieuw bestand na elke batch van 50 soorten
#             batch_file = os.path.join(batch_directory, f"observations_batch_{batch_count}.json")
#             with open(batch_file, "w", encoding="utf-8") as f:
#                 json.dump(all_observations['observations'], f, indent=4, ensure_ascii=False)
            
#             print(f"Opgeslagen batch {batch_count} met {len(all_observations['observations'])} observaties.")
            
#             # Reset de observaties lijst voor de volgende batch
#             all_observations['observations'] = []
#             batch_count += 1
#             species_in_batch = 0  # Reset het aantal soorten in de batch

#         # Geen stopconditie meer op basis van het aantal soorten
#         # Als je de loop wilt beëindigen op een ander moment, kun je dit hier aanpassen

#     # Wacht voor de laatste threads om klaar te zijn
#     for t in threads:
#         t.join()

#     # Sla alle resterende observaties op
#     if all_observations['observations']:
#         batch_file = os.path.join(batch_directory, f"observations_batch_{batch_count}.json")
#         with open(batch_file, "w", encoding="utf-8") as f:
#             json.dump(all_observations['observations'], f, indent=4, ensure_ascii=False)

#     print(f"Opgeslagen {len(all_observations['observations'])} observaties.")

# # Roep de functie aan om de gegevens op te halen
# fetch_all_observations()


Als ik door ga scrapen kan ik opnieuw beginnen, maar ben nu getspt bij 11600 soorten

In [1]:
import requests
import json
import threading
import time
import os  # Voor het maken van directories
from datetime import datetime

start_data = "2023-01-01"
end_date = "2025-12-31"
Location = "Rotterdam"
time_sleep = 5
species_per_batch = 50  # Aantal soorten per batch

# Genereer een unieke directory naam met een timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
batch_directory = f"batches_{timestamp}"

# Maak de directory aan
os.makedirs(batch_directory, exist_ok=True)

# Het species_id vanaf waar je wilt starten
start_species_id = 1  


# Functie om observaties op te halen voor een bepaald soort
def fetch_observations_for_species(species_id, results):
    API_URL = f"https://observation.org/api/v1/species/{species_id}/observations/?limit=1000&country_id=166&date_after={start_data}&date_before={end_date}&user=&location={Location}"
    
    retries = 5  # Maximaal aantal herhalingen bij een 429-fout
    while retries > 0:
        response = requests.get(API_URL)

        if response.status_code == 200:
            data = response.json()
            
            # Controleer of er observaties zijn
            if 'detail' in data and data['detail'] == "No Species matches the given query.":
                print(f"Geen observaties voor soort {species_id}.")
                return
            
            # Voeg de resultaten toe
            with results['lock']:
                results['observations'].extend(data.get("results", []))
            
            time.sleep(time_sleep)  # Rate-limit bescherming
            return
        
        elif response.status_code == 429:
            print(f"Rate limit bereikt voor soort {species_id}, opnieuw proberen in 5 seconden...")
            time.sleep(time_sleep)
            retries -= 1  # Aantal pogingen verlagen
        
        else:
            print(f"Fout bij ophalen voor soort {species_id}: {response.status_code}")
            return  # Stop als er een andere fout is

# Functie die de threading afhandelt
def fetch_all_observations():
    all_observations = {'observations': [], 'lock': threading.Lock()}
    threads = []
    species_id = start_species_id
    batch_count = 1
    species_in_batch = 0
    
    while True:
        print(f"Verwerken voor soort {species_id}...")
        thread = threading.Thread(target=fetch_observations_for_species, args=(species_id, all_observations))
        threads.append(thread)
        thread.start()
        
        species_id += 1
        species_in_batch += 1
        
        # Wacht totdat 15 threads klaar zijn of 50 soorten verwerkt zijn
        if len(threads) >= 10 or species_in_batch >= species_per_batch:
            for t in threads:
                t.join()  
            threads = []

            # Opslaan van batch
            batch_file = os.path.join(batch_directory, f"observations_batch_{batch_count}.json")
            with open(batch_file, "w", encoding="utf-8") as f:
                json.dump(all_observations['observations'], f, indent=4, ensure_ascii=False)
            
            print(f"Opgeslagen batch {batch_count} met {len(all_observations['observations'])} observaties.")
            
            # Reset
            all_observations['observations'] = []
            batch_count += 1
            species_in_batch = 0  

    # Wacht op de laatste threads
    for t in threads:
        t.join()

    # Sla de laatste batch op
    if all_observations['observations']:
        batch_file = os.path.join(batch_directory, f"observations_batch_{batch_count}.json")
        with open(batch_file, "w", encoding="utf-8") as f:
            json.dump(all_observations['observations'], f, indent=4, ensure_ascii=False)

    print(f"Opgeslagen {len(all_observations['observations'])} observaties.")

# Start het ophalen van data
fetch_all_observations()


Verwerken voor soort 1...
Verwerken voor soort 2...
Verwerken voor soort 3...
Verwerken voor soort 4...
Verwerken voor soort 5...
Verwerken voor soort 6...
Verwerken voor soort 7...
Verwerken voor soort 8...
Verwerken voor soort 9...
Verwerken voor soort 10...
Fout bij ophalen voor soort 9: 404
Opgeslagen batch 1 met 1788 observaties.
Verwerken voor soort 11...
Verwerken voor soort 12...
Verwerken voor soort 13...
Verwerken voor soort 14...
Verwerken voor soort 15...
Verwerken voor soort 16...
Verwerken voor soort 17...
Verwerken voor soort 18...
Verwerken voor soort 19...
Verwerken voor soort 20...
Opgeslagen batch 2 met 107 observaties.
Verwerken voor soort 21...
Verwerken voor soort 22...
Verwerken voor soort 23...
Verwerken voor soort 24...
Verwerken voor soort 25...
Verwerken voor soort 26...
Verwerken voor soort 27...
Verwerken voor soort 28...
Verwerken voor soort 29...
Verwerken voor soort 30...
Opgeslagen batch 3 met 815 observaties.
Verwerken voor soort 31...
Verwerken voor s

KeyboardInterrupt: 